In [ ]:
# Import libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
import tkinter as tk
from tkinter import filedialog

print(f"TensorFlow version: {tf.__version__}")

# Check dataset
dataset_path = './Dataset'
if os.path.exists(dataset_path):
    classes = os.listdir(dataset_path)
    print(f"Dataset found with classes: {classes}")
else:
    print("Dataset not found!")

In [ ]:
# Data preparation
train_dir = './Dataset'
img_size = (64, 64)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Build model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model built successfully")

In [ ]:
# Train model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

final_train_accuracy = history.history['accuracy'][-1] * 100
final_val_accuracy = history.history['val_accuracy'][-1] * 100
print(f"Training Accuracy: {final_train_accuracy:.2f}%")
print(f"Validation Accuracy: {final_val_accuracy:.2f}%")

In [ ]:
# Save model and get class names
model.save('flower_classifier_model.h5')
class_names = list(train_generator.class_indices.keys())
print(f"Model saved. Classes: {class_names}")

In [ ]:
# Plot training results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Image selection and prediction function
def select_and_predict_image():
    root = tk.Tk()
    root.withdraw()
    
    img_path = filedialog.askopenfilename(
        title="Select a flower image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
    )
    
    if img_path:
        try:
            # Load and preprocess image
            img = image.load_img(img_path, target_size=img_size)
            img_array = image.img_to_array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            
            # Make prediction
            prediction = model.predict(img_array, verbose=0)
            predicted_class = class_names[np.argmax(prediction)]
            confidence = np.max(prediction) * 100
            
            # Display results
            plt.figure(figsize=(10, 4))
            
            # Show image
            plt.subplot(1, 2, 1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f'Selected Image\\nPredicted: {predicted_class} ({confidence:.1f}%)')
            
            # Show probabilities
            plt.subplot(1, 2, 2)
            probabilities = prediction[0] * 100
            colors = ['green' if i == np.argmax(prediction) else 'lightblue' for i in range(len(class_names))]
            bars = plt.bar(class_names, probabilities, color=colors)
            plt.ylabel('Confidence (%)')
            plt.title('Prediction Confidence')
            plt.ylim(0, 100)
            
            for bar, prob in zip(bars, probabilities):
                plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                        f'{prob:.1f}%', ha='center', va='bottom')
            
            plt.tight_layout()
            plt.show()
            
            print(f"Predicted flower: {predicted_class} ({confidence:.1f}% confidence)")
            
        except Exception as e:
            print(f"Error processing image: {str(e)}")
    else:
        print("No image selected")
    
    root.destroy()

print("Image selection function ready")

In [ ]:
# Test with your own image
select_and_predict_image()